In [1]:
import pandas as pd
import numpy as np

import neologdn
import MeCab

import re

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

# bag of wordsを作成するためのライブラリ
from gensim import corpora, matutils
# TF-IDFを作成するためのライブラリ
from gensim import models

C:\Users\Tomori kengo\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### サンプルデータの読み込み

In [2]:
#encoding='utf-8'で上手くいった
diary_df = pd.read_csv('SBI_Financial statement_201903.csv', encoding='utf-8')

In [3]:
diary_df

,text
0,国内経済が緩やかに回復するなか、マーケット環境は、米国と中国の貿易摩擦問題に対する警戒感等か...
1,このような環境の中、当社業績においては、ホールセールビジネスの拡大、トレーディング収益や金融...
2,当社は引続き他社を大きく上回る高いシェアを維持し、35.9％のシェアを獲得。
3,先物・オプションの委託個人売買代金シェアは、引き続き高水準を維持。
4,投資信託残高の四半期末残高は過去最高を更新し、信託報酬は高水準を維持。
5,2018年4月から2018年6月までの上場会社数は20社。
6,同期間のSBI証券引受関与率は100％と 引き続き業界トップ。


### MeCabにかける前準備としてneologdn.normalize()を使用して文章全体を正規化

In [4]:
def get_diary_normalization(text):
    diary_normalization = neologdn.normalize(text)
    return diary_normalization

### MeCab + neologdで形態素解析し、名詞、形容詞原形と動詞原形を抽出しリストに格納
##### 参考：https://github.com/kujirahand/book-mlearn-gyomu/blob/master/src/ch4/Doc2Vec/create_model.py

In [5]:
neologd_tagger = MeCab.Tagger('-Ochasen -d C:\mecab-ipadic-neologd')

# 引数のテキストを分かち書きして配列にする
# node.surface: 文字のみ取得できる 出力例：同  期間  の  SBI  証券
# node.feature: 品詞、原形などの詳細を取得できる:「品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用形,活用型,原形,読み,発音」の順
# よって、「品詞」を取得したい場合はnode.featureをsplit()でリスト型にした後に抽出したい詳細のインデックス番号[0]を指定すればよい
# また、動詞や形容詞の「原形（の単語）」を取得したい場合はインデックス番号[6]を指定すればよい

def split_words(diary_normalization):
    node = neologd_tagger.parseToNode(diary_normalization) #parseだとエラー「 'str' object has no attribute 'feature'」
    wakati_words = []
    while node is not None:# nodeに何か格納されていれば処理を継続する
        hinshi = node.feature.split(",")[0]# splitでリスト型に変換し、そのリストの0番目(=品詞)を抽出
        if  hinshi in ["名詞"]:
            wakati_words.append(node.surface)# hinshiが名詞であれば単語そのものをwakati_wordsに追加
        elif hinshi in ["動詞", "形容詞"]:
            wakati_words.append(node.feature.split(",")[6])# hinshiが動詞or形容詞であれば単語の原形をwakati_wordsに追加
        node = node.next
    return wakati_words

In [6]:
'''
wakati_list = []

# 文章の上から順に作成した関数を実行
for diary in diary_df['text']:
    diary_normalization = get_diary_normalization(diary)
    wakati_words = split_words(diary_normalization)
    wakati_list.append(wakati_words)
    
wakati_list
'''

"\nwakati_list = []\n\n# 文章の上から順に作成した関数を実行\nfor diary in diary_df['text']:\n    diary_normalization = get_diary_normalization(diary)\n    wakati_words = split_words(diary_normalization)\n    wakati_list.append(wakati_words)\n    \nwakati_list\n"

### 素性ベクトル（文章の特徴を表現したベクトル）への変換

In [6]:
# diary_dfのtextカラムのすべての文章に「split_words」関数（形態素解析する関数）を適応させたい

# apply関数の引数に関数を引き渡すことが可能、関数を全列に適応できる
#  参考URL：https://teratail.com/questions/134846
# diary_dfに関数を全列新たなカラム['Wakati']を追加する
# 参考URL：https://datumstudio.jp/blog/%e3%80%90%e7%89%b9%e5%88%a5%e9%80%a3%e8%bc%89%e3%80%91-%e3%81%95%e3%81%81%e3%80%81%e8%87%aa%e7%84%b6%e8%a8%80%e8%aa%9e%e5%87%a6%e7%90%86%e3%82%92%e5%a7%8b%e3%82%81%e3%82%88%e3%81%86%ef%bc%81-3
diary_df['Wakati'] = diary_df['text'].apply(split_words)

In [7]:
diary_df

,text,Wakati
0,国内経済が緩やかに回復するなか、マーケット環境は、米国と中国の貿易摩擦問題に対する警戒感等か...,"[国内, 経済, 緩やか, 回復, する, なか, マーケット, 環境, 米国, 中国, 貿..."
1,このような環境の中、当社業績においては、ホールセールビジネスの拡大、トレーディング収益や金融...,"[よう, 環境, 中, 当社, 業績, ホールセール, ビジネス, 拡大, トレーディング,..."
2,当社は引続き他社を大きく上回る高いシェアを維持し、35.9％のシェアを獲得。,"[当社, 他社, 大きい, 上回る, 高い, シェア, 維持, する, 35, 9, ％, ..."
3,先物・オプションの委託個人売買代金シェアは、引き続き高水準を維持。,"[先物, オプション, 委託, 個人, 売買代金, シェア, 高水準, 維持]"
4,投資信託残高の四半期末残高は過去最高を更新し、信託報酬は高水準を維持。,"[投資信託, 残高, 四半期, 末, 残高, 過去最高, 更新, する, 信託報酬, 高水準..."
5,2018年4月から2018年6月までの上場会社数は20社。,"[2018年, 4月, 2018年, 6月, 上場会社, 数, 20, 社]"
6,同期間のSBI証券引受関与率は100％と 引き続き業界トップ。,"[期間, SBI証券, 引受, 関与, 率, 100％, 業界, トップ]"


In [8]:
# Wakatiカラムの1行毎に格納されている単語はリストに格納されているのでリスト型と判断されCountVectorizer.fit_transform()が使用できない
# そこでjoin関数でリストの中身を文字列に変換する
# ただし、リストに格納されてる要素に数値が混ざってる場合はmap([適用関数], [対象リスト])を使う必要あり
# 参考URL：https://www.okadalabo.com/python%E3%83%AA%E3%82%B9%E3%83%88%E3%81%AE%E4%B8%AD%E8%BA%AB%E3%82%92%E6%96%87%E5%AD%97%E5%88%97%E3%81%AB%E5%A4%89%E6%8F%9B%E3%81%99%E3%82%8B%E3%80%82/
for i in range(0, len(diary_df)):
    diary_df['Wakati'][i] = ','.join((map(str, diary_df['Wakati'][i] )))

In [9]:
diary_df

,text,Wakati
0,国内経済が緩やかに回復するなか、マーケット環境は、米国と中国の貿易摩擦問題に対する警戒感等か...,"国内,経済,緩やか,回復,する,なか,マーケット,環境,米国,中国,貿易摩擦,問題,警戒感,..."
1,このような環境の中、当社業績においては、ホールセールビジネスの拡大、トレーディング収益や金融...,"よう,環境,中,当社,業績,ホールセール,ビジネス,拡大,トレーディング,収益,金融,収益,..."
2,当社は引続き他社を大きく上回る高いシェアを維持し、35.9％のシェアを獲得。,"当社,他社,大きい,上回る,高い,シェア,維持,する,35,9,％,シェア,獲得"
3,先物・オプションの委託個人売買代金シェアは、引き続き高水準を維持。,"先物,オプション,委託,個人,売買代金,シェア,高水準,維持"
4,投資信託残高の四半期末残高は過去最高を更新し、信託報酬は高水準を維持。,"投資信託,残高,四半期,末,残高,過去最高,更新,する,信託報酬,高水準,維持"
5,2018年4月から2018年6月までの上場会社数は20社。,"2018年,4月,2018年,6月,上場会社,数,20,社"
6,同期間のSBI証券引受関与率は100％と 引き続き業界トップ。,"期間,SBI証券,引受,関与,率,100％,業界,トップ"


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer()
# CountVectorizerを利用して単語と列番号の対応付けを実行 Document-Term Matrixを獲得できる
feature_vectors = CV.fit_transform(diary_df['Wakati'])
# どの単語を学習しているのかをCV.get_feature_names()で確認
vocabulary = CV.get_feature_names()

In [11]:
print(feature_vectors)

  (0, 35)	1
  (0, 53)	1
  (0, 51)	1
  (0, 26)	1
  (0, 52)	1
  (0, 19)	1
  (0, 47)	1
  (0, 46)	1
  (0, 16)	1
  (0, 8)	1
  (0, 7)	1
  (0, 28)	1
  (0, 50)	1
  (0, 48)	1
  (0, 29)	1
  (1, 55)	1
  (1, 31)	1
  (1, 24)	1
  (1, 30)	1
  (1, 56)	1
  (1, 25)	2
  (1, 13)	1
  (1, 39)	1
  (1, 14)	1
  (1, 15)	1
  :	:
  (3, 10)	1
  (3, 23)	1
  (3, 49)	1
  (3, 11)	1
  (4, 21)	1
  (4, 40)	1
  (4, 54)	1
  (4, 27)	1
  (4, 44)	2
  (4, 38)	1
  (4, 59)	1
  (4, 49)	1
  (4, 7)	1
  (5, 1)	1
  (5, 18)	1
  (5, 5)	1
  (5, 4)	1
  (5, 2)	2
  (6, 12)	1
  (6, 42)	1
  (6, 0)	1
  (6, 57)	1
  (6, 36)	1
  (6, 6)	1
  (6, 41)	1


In [12]:
print('今回の文章で学習した単語の数は{}語です。'.format(len(CV.get_feature_names())))
# print('今回の文章で学習した単語の数は{}語です。'.format(len(CV.vocabulary_.keys()))) 上記はこの表現でも同じ
print('--------------------------------------------------------------------------------------------')
print(vocabulary)
print('--------------------------------------------------------------------------------------------')
print(CV.vocabulary_)
print('--------------------------------------------------------------------------------------------')
print(CV.vocabulary_.keys())
print('--------------------------------------------------------------------------------------------')
print(CV.vocabulary_.values())

今回の文章で学習した単語の数は60語です。
--------------------------------------------------------------------------------------------
['100', '20', '2018年', '35', '4月', '6月', 'sbi証券', 'する', 'なか', 'よう', 'オプション', 'シェア', 'トップ', 'トレーディング', 'ビジネス', 'ホールセール', 'マーケット', '上回る', '上場会社', '中国', '他社', '信託報酬', '個人', '先物', '前年同期', '収益', '問題', '四半期', '回復', '国内', '増加', '増収増益', '売買代金', '大きい', '委託', '展開', '引受', '当社', '投資信託', '拡大', '更新', '期間', '業界', '業績', '残高', '獲得', '環境', '米国', '経済', '維持', '緩やか', '警戒感', '貿易摩擦', '退る', '過去最高', '達成', '金融', '関与', '高い', '高水準']
--------------------------------------------------------------------------------------------
{'国内': 29, '経済': 48, '緩やか': 50, '回復': 28, 'する': 7, 'なか': 8, 'マーケット': 16, '環境': 46, '米国': 47, '中国': 19, '貿易摩擦': 52, '問題': 26, '警戒感': 51, '退る': 53, '展開': 35, 'よう': 9, '当社': 37, '業績': 43, 'ホールセール': 15, 'ビジネス': 14, '拡大': 39, 'トレーディング': 13, '収益': 25, '金融': 56, '増加': 30, '前年同期': 24, '増収増益': 31, '達成': 55, '他社': 20, '大きい': 33, '上回る': 17, '高い': 58, 'シェア': 11, '維持': 49, '35': 3, '獲得': 45, '

In [13]:
print(feature_vectors.toarray())

[[0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1
  0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 2 0 0 0 0 1 1 0 0 0 0
  0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 2 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
  0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0
  0 0 1 0 1 0 0 0 2 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1]
 [0 1 2 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]]


In [14]:
# 疎行列をデータに、インデックスを元のテキストに、カラムを「文章から抽出した単語」したデータフレームを作成
# 書籍「Pythonによるあたらしいデータ分析の教科書 P.290」が参考になるかも
# columns=CV.vocabulary_.keys()にすると単語の並びとfeature_vectors.toarray()の0,1の並びとが対応できていないのでcolumnsにはCV.get_feature_names()を指定する
bow_df = pd.DataFrame(feature_vectors.toarray(), index=diary_df['text'],  columns=vocabulary)

In [15]:
bow_df

,100,20,2018年,35,4月,6月,sbi証券,する,なか,よう,...,緩やか,警戒感,貿易摩擦,退る,過去最高,達成,金融,関与,高い,高水準
text,,,,,,,,,,,,,,,,,,,,,
国内経済が緩やかに回復するなか、マーケット環境は、米国と中国の貿易摩擦問題に対する警戒感等から一進一 退の展開。,0,0,0,0,0,0,0,1,1,0,...,1,1,1,1,0,0,0,0,0,0
このような環境の中、当社業績においては、ホールセールビジネスの拡大、トレーディング収益や金融 収益の増加により前年同期比で増収増益を達成。,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,1,0,0,0
当社は引続き他社を大きく上回る高いシェアを維持し、35.9％のシェアを獲得。,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
先物・オプションの委託個人売買代金シェアは、引き続き高水準を維持。,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
投資信託残高の四半期末残高は過去最高を更新し、信託報酬は高水準を維持。,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
2018年4月から2018年6月までの上場会社数は20社。,0,1,2,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
同期間のSBI証券引受関与率は100％と 引き続き業界トップ。,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [16]:
# to_csv関数でindexにカラム名を名付けるには「index_label='カラム名'」 を引数に追加する
bow_df.to_csv('bow_df.csv', index_label='text', encoding='shift-jis')